In [2]:
import pandas as pd
import numpy as np
import math


In [3]:
complete_orders=pd.read_csv("../data/nb.csv")
delivery_requests=pd.read_csv("../data/driver_locations_during_request.csv")

information about the completed orders

In [5]:
complete_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536020 entries, 0 to 536019
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Trip ID           536020 non-null  int64 
 1   Trip Origin       536020 non-null  object
 2   Trip Destination  536020 non-null  object
 3   Trip Start Time   534369 non-null  object
 4   Trip End Time     536019 non-null  object
dtypes: int64(1), object(4)
memory usage: 20.4+ MB


In [20]:
complete_orders.isna().sum()

Trip ID                0
Trip Origin            0
Trip Destination       0
Trip Start Time     1651
Trip End Time          1
dtype: int64

We only have one missing value for Trip End Time, let's check if Trip Start Time is also null for that specific record

In [8]:
complete_orders[complete_orders["Trip End Time"].isna()]

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time
59737,530900,"6.4971445,3.3384259","6.529695343361617,3.386906603181526",2021-07-22 19:54:15,NaN


since we have the Trip start time value, we can use average trip duration to fill the missing values (fro both start & end time)

In [55]:
# First we have to change the columns to appropriate datatype : date

def change_to_date(df:pd.DataFrame,columns:list):
    for column in columns:
        df[column] = pd.to_datetime(df[column])
    return df

In [31]:
def calculate_duration(df:pd.DataFrame,start_col_name,end_col_name):
    df["duration"]= (df[end_col_name] - df[start_col_name]).astype('timedelta64[m]')
    return df

In [60]:
complete_orders_correct_format=complete_orders.copy()
complete_orders_correct_format=change_to_date(complete_orders_correct_format,["Trip Start Time","Trip End Time"])
complete_orders_correct_format.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536020 entries, 0 to 536019
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Trip ID           536020 non-null  int64         
 1   Trip Origin       536020 non-null  object        
 2   Trip Destination  536020 non-null  object        
 3   Trip Start Time   534369 non-null  datetime64[ns]
 4   Trip End Time     536019 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(2)
memory usage: 20.4+ MB


In [65]:
# drop
complete_orders_correct_format=calculate_duration(complete_orders_correct_format,"Trip Start Time","Trip End Time")
complete_orders_correct_format.isna().sum()

Trip ID                0
Trip Origin            0
Trip Destination       0
Trip Start Time     1651
Trip End Time          1
duration            1652
dtype: int64

In [8]:
complete_orders.head()

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36


In [19]:
percentage = round(complete_orders.nunique()*100/complete_orders.shape[0],2)
pd.DataFrame(percentage,columns=["Unique %"])

,Unique %
Trip ID,100.00
Trip Origin,20.92
Trip Destination,27.52
Trip Start Time,95.62
Trip End Time,95.86


In [36]:
print(complete_orders[complete_orders["Trip Origin"]==(0,0)].shape)
print(complete_orders[complete_orders["Trip Destination"]==(0,0)].shape)

(0, 5)
(0, 5)


In [9]:
delivery_requests.head()

,id,order_id,driver_id,driver_action,lat,lng,created_at,updated_at
0,1,392001,243828,accepted,6.602207,3.270465,NaN,NaN
1,2,392001,243588,rejected,6.592097,3.287445,NaN,NaN
2,3,392001,243830,rejected,6.596133,3.281784,NaN,NaN
3,4,392001,243539,rejected,6.596142,3.280526,NaN,NaN
4,5,392001,171653,rejected,6.609232,3.288800,NaN,NaN


In [16]:
null_trip=list(complete_orders[complete_orders["Trip Start Time"].isna()]["Trip ID"])
# delivery_requests[delivery_requests["order_id"] in null_trip]
delivery_requests[pd.DataFrame(delivery_requests.order_id.tolist()).isin(null_trip).any(1).values]

C:\Users\Natnael\AppData\Local\Temp\ipykernel_8792\2632293708.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  delivery_requests[pd.DataFrame(delivery_requests.order_id.tolist()).isin(null_trip).any(1).values]


,id,order_id,driver_id,driver_action,lat,lng,created_at,updated_at
51887,51888,394447,243773,accepted,6.435905,3.493106,NaN,NaN
51888,51889,394447,243863,rejected,6.434726,3.500062,NaN,NaN
51889,51890,394447,244109,rejected,6.433196,3.503248,NaN,NaN
51890,51891,394447,243966,rejected,6.430786,3.502685,NaN,NaN
51891,51892,394447,243827,rejected,6.437021,3.508620,NaN,NaN
...,...,...,...,...,...,...,...,...
1542519,1542520,447888,244095,rejected,6.447715,3.409985,NaN,NaN
1542520,1542521,447888,245658,rejected,6.436035,3.409957,NaN,NaN
1542521,1542522,447888,243059,rejected,6.453392,3.430967,NaN,NaN
1542522,1542523,447888,243217,rejected,6.436125,3.430194,NaN,NaN
